In [1]:
from openbb_terminal.api import openbb
from company import Company

from IPython.display import display


WARNING! Breaking changes incoming! Especially avoid using kwargs, since some of them will change.
For more information see the official documentation at: https://openbb-finance.github.io/OpenBBTerminal/api/


In [2]:
# https://developer.twitter.com
# API_TWITTER_KEY = "cl1F94kpMHTrkLGasn2IxUNoh"
# API_TWITTER_SECRET_KEY = "AdPl5u6aavlymJ6L7uCyuyzRIIpXHGOKs0btqs2Abu4876BAFb"
# API_TWITTER_BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAKffhwEAAAAAuCtW%2FeTKWuX35ZzFvKTFUKBvOq0%3DHmO1Shm5Fsa8u90G9MLh5ql1LmSPTF8xrMtrOJSl81gnLpOjV0"


In [3]:
peers = [('MSFT', 1), ('DIS', 2), ('VEEV', 3), ('TDOC', 4)]
# , ('TDOC', 3), ('VEEV', 4)]

for company in peers:
    
    stock = company[0]
    slot = company[1]

##################################################################################
############## *** setting variables that hold dataframes: *** ###################
##################################################################################
    df_balance = openbb.stocks.fa.yf_financials(stock, "balance-sheet") 
    df_data = openbb.stocks.fa.data(stock) # keys called later: 'Profit Margin', 'Insider Own', 'Insider Trans', 'Short Float', 'Inst Own', 'Inst Trans', 'Dividend'
    df_divs = openbb.stocks.fa.divs(stock)
    df_income = openbb.stocks.fa.fmp_income(stock) # keys called later: 'Gross profit ratio'
    
    df_metrics = openbb.stocks.fa.fmp_metrics(stock) # keys called later: 'Book value per share', 'Ptb ratio', 'Pe ratio', 
                                                                        # 'Pfcf ratio', 'Dividend yield', 'Roe', 'Return on tangible assets', 
                                                                        # 'Current ratio', 'Debt to equity', 'Price to sales ratio'
            
    df_profile = openbb.stocks.fa.profile(stock) # keys called later: 'companyName', 'sector', 'industry' 
    df_quote = openbb.stocks.fa.quote(stock) # keys called later: 'Price', 'Market cap'

    
##################################################################################
############## *** setting variables that we'll add to company objects: *** ######
##################################################################################   
          
    
############## *** BASIC DETAILS *** ##############

    fullname = df_profile.loc['companyName'][0]
    ticker = stock
    cap = df_quote.loc['Market cap'][0]
    price = df_quote.loc['Price'][0]

############## *** Value Metrics *** ##############
   
    try: # If company has no long term debt 0 will be assigned to associated variable
        tca = df_balance.loc['Total current assets'][0]
        tld = df_balance.loc['Long-term debt'][0]
        
    except:
        tca = df_balance.loc['Total current assets'][0]
        tld = 0     

    ptb = df_metrics.loc['Ptb ratio'][0]
    bvps = df_metrics.loc['Book value per share'][0]
    pe = df_metrics.loc['Pe ratio'][0]
    pfcf = df_metrics.loc['Pfcf ratio'][0]
    pts = df_metrics.loc['Price to sales ratio'][0]

############## *** Management Metrics *** ##############

    roe = df_metrics.loc['Roe'][0]
    roa = df_metrics.loc['Return on tangible assets'][0]
    gpr = df_income.loc['Gross profit ratio'][0]
    
    try: # Some companies are do not have profit margin
        pm = df_data.loc['Profit Margin'][0]
        
    except:
        pm = '0%'

    cr = df_metrics.loc['Current ratio'][0]
    dte = df_metrics.loc['Debt to equity'][0]
        
############## *** Insider Metrics *** ##############

    io = df_data.loc['Insider Own'][0]
    it = df_data.loc['Insider Trans'][0]
    inst_o = df_data.loc['Inst Own'][0]
    inst_t = df_data.loc['Inst Trans'][0]
    
############## *** Dividend Metrics *** ##############

    try: # If company provides no dividend 0 will be assigned
        div = df_data.loc['Dividend'][0]
    
    except:
        div = 0
   
    try: # If company provides no dividend 0 will be assigned
        div_y = df_metrics.loc['Dividend yield'][0]
        
    except:
        div_y = 0
        
    div_h = df_divs
    

        
        
        




    # using df_profile to get companyName used for search string
    df_news = openbb.common.news(df_profile.loc['companyName', 0], sort='published')



    # s_analyst = openbb.stocks.dd.analyst("MSFT")
    #s_analyst = s_analyst.T
    # s_analyst = s_analyst.loc['rating']

    s_rot = openbb.stocks.dd.rot(stock)
    s_rot = s_rot.T
    s_rot = s_rot[0]

    s_key = openbb.stocks.fa.av_metrics(stock)
    s_key = s_key.loc['Dividend yield']

    
    # Getting Stock Twits sentiment as a float:
    f_twits = openbb.stocks.ba.bullbear(stock)
    
    if f_twits[1] != 0 and f_twits[2] != 0: # Normal case
        f_twits_percent = f_twits[2] / f_twits[1]
        
    elif f_twits[1] == 0: # Case for if there are no ratings
        f_twits_percent = 'Not Rated'
        
    elif f_twits[2] == 0: # Case for if all ratings are bearish
        f_twits_percent = 0

    # Warren Buffet Investing Score
    wb_score = openbb.stocks.fa.score(stock)
    if wb_score is None:
        wb_score = 'n/a'

    # creating ratings list
    s_rating = openbb.stocks.dd.rating(stock)
    s_rating = s_rating.T
    s_rating = s_rating.loc['Rating']

    ratings_list = []
    for i in range(0, 5):
        ratings_list.append(s_rating[i])

    if slot == 1:

        company1 = Company(fullname, ticker, cap, price,
                           tca, tld, ptb, bvps, pe, pfcf, pts,
                           roe, roa, gpr, pm, cr, dte,
                           io, it, inst_o, inst_t,
                           div, div_y, div_h)

    elif slot == 2:

        company2 = Company(fullname, ticker, cap, price,
                           tca, tld, ptb, bvps, pe, pfcf, pts,
                           roe, roa, gpr, pm, cr, dte,
                           io, it, inst_o, inst_t,
                           div, div_y, div_h)
        
    elif slot == 3:

        company3 = Company(fullname, ticker, cap, price,
                           tca, tld, ptb, bvps, pe, pfcf, pts,
                           roe, roa, gpr, pm, cr, dte,
                           io, it, inst_o, inst_t,
                           div, div_y, div_h)
        
    elif slot == 4:

        company4 = Company(fullname, ticker, cap, price,
                           tca, tld, ptb, bvps, pe, pfcf, pts,
                           roe, roa, gpr, pm, cr, dte,
                           io, it, inst_o, inst_t,
                           div, div_y, div_h)

Fetching data. Please be patient

Fetching data. Please be patient

Fetching data. Please be patient

Ticker should be a NASDAQ 100 ticker or SP 500 ticker

Fetching data. Please be patient

Ticker should be a NASDAQ 100 ticker or SP 500 ticker

In [4]:
print(company1.fullname, '   ', company1.div_h)
print(company2.fullname, '   ', company2.div_h)
print(company3.fullname, '   ', company3.div_h)
print(company4.fullname, '   ', company4.div_h)

Microsoft Corporation                 Dividends
Date                 
2003-02-19       0.08
2003-10-15       0.16
2004-08-23       0.08
2004-11-15       3.08
2005-02-15       0.08
...               ...
2021-08-18       0.56
2021-11-17       0.62
2022-02-16       0.62
2022-05-18       0.62
2022-08-17       0.62

[75 rows x 1 columns]
The Walt Disney Company                 Dividends
Date                 
1962-03-13   0.000249
1962-06-12   0.000249
1962-09-11   0.000249
1962-12-18   0.000257
1963-03-12   0.000257
...               ...
2017-12-08   0.840000
2018-07-06   0.840000
2018-12-07   0.880000
2019-07-05   0.880000
2019-12-13   0.880000

[124 rows x 1 columns]
Veeva Systems Inc.     Empty DataFrame
Columns: [Dividends]
Index: []
Teladoc Health, Inc.     Empty DataFrame
Columns: [Dividends]
Index: []


In [ ]:
openbb.stocks.fa.divs('TDOC')

In [ ]:
def display_df(dataframes):
    for df in dataframes:
        display(df)
        print('\n')

def display_s(list_of_series):
    for series in list_of_series:
        display(series)
        print('\n')

def display_f(forumlaic_value):
    for value in forumlaic_value:
        print(str(value) + '\n')
    


In [ ]:
dataframes = [df_quote, df_data, df_metrics, df_balance, df_divs, df_profile] 

series = [s_income, s_rating, s_rot]

formulaic = [f_twits_percent]




In [ ]:
display_df(dataframes)
display_s(series)
display_f(formulaic)
ratings_list
wb_score

In [ ]:
s_key